In [1]:
import sys, string
import os
import socket
import time
from time import gmtime, strftime
import operator
import boto3
import json
import findspark
from pyspark.sql import SparkSession
from datetime import datetime


In [2]:
findspark.init()

In [3]:
spark = SparkSession\
        .builder\
        .appName("ethereum-sneha")\
        .getOrCreate()

In [4]:
def good_contract_line(line):
    try:
        fields = line.split(',')
        # print(len(fields))
        # print(fields[0])
        if len(fields) != 5 or fields[0] == 'address':
            return False
        else:
            return True
        # int(feilds[6])
        # return True
    except:
        return False


def good_trans_line(line):
    try:
        fields = line.split(',')
        # print(len(fields))
        # print(fields[0])
        if len(fields) != 7 or fields[0] == 'block_number':
            return False
        else:
            return True
        int(feilds[6])
        # return True
    except:
        return False



In [5]:
contracts_file = spark.sparkContext.textFile("./contract.csv")


In [6]:
clean_contracts = contracts_file.filter(good_contract_line)

In [7]:
contracts_address = clean_contracts.map(lambda x: (x.split(',')[0], 1))

In [8]:
# transaction_file.collect()[:5]
transaction_file = spark.sparkContext.textFile("./transactions.csv")
clean_transactions = transaction_file.filter(good_trans_line)


In [9]:
transactions_sent_ether_address = clean_transactions.map(lambda y: (y.split(',')[2], float(y.split(',')[3])))

In [10]:
transactions_to_address = transactions_sent_ether_address.reduceByKey(operator.add)

In [11]:
transaction_contracts_join = transactions_to_address.join(contracts_address)

In [12]:
top_10_contracts = transaction_contracts_join.takeOrdered(10, key = lambda z: -z[1][0])


In [13]:
# for record in top_10_contracts:
#     print("{}: {};{}".format(record[0], record[1][0], record[1][1]))
top_10_contracts

[('0x996035cae22cd90ae5e993d0832cfac4db239109', (4477970000000000.0, 1)),
 ('0xcd9a70f197238f0ee7045080368dc7d07502b353', (2100000000000000.0, 1))]

In [14]:
clean_transactions.collect()[:2]

['2280135,0x44130614587f420c4019b7ae9fbf763982cdf2fe,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,1.00001816E+018,132650,20000000000,1474174806',
 '2280135,0x5af3fa46441acdc8048cff7b5c00b1fb89ffa752,0x5845b8f77109be22c93b4b75a3ad02283b7cc9d8,3.77792107478E+017,21000,20000000000,1474174806']

In [15]:
clean_contracts.collect()[:2]

['0x9a78bba29a2633b4b3162a28ce498c98daee5af8,false,false,8623545,2019-09-26 08:50:33 UTC',
 '0x85aa7fbc06e3f959aa358159db60e063171314db,false,false,8621323,2019-09-26 00:29:37 UTC']

In [16]:
contracts_address.collect()[:2]

[('0x9a78bba29a2633b4b3162a28ce498c98daee5af8', 1),
 ('0x85aa7fbc06e3f959aa358159db60e063171314db', 1)]

In [17]:
transactions_to_address.collect()[:5]

[('0x8b3b35fd0bfdaba763174771b310839e542acd18', 1.0821151e+18),
 ('0x94f3a2d4bd7ae87ca889ff905303671b5999f1e9', 1.08119166e+18),
 ('0xd2408df325d776bd922808d975b36cd0cc469f26', 1.00170607593967e+18),
 ('0xfdc77b9cb732eb8c896b152e28294521f5f62e67', 0.0),
 ('0x5bc109a431bc1c9b654c33270350048fd26b789a', 1.07094254e+18)]

In [18]:
transaction_contracts_join.collect()

[('0x996035cae22cd90ae5e993d0832cfac4db239109', (4477970000000000.0, 1)),
 ('0xcd9a70f197238f0ee7045080368dc7d07502b353', (2100000000000000.0, 1))]

In [19]:
top_10_contracts

[('0x996035cae22cd90ae5e993d0832cfac4db239109', (4477970000000000.0, 1)),
 ('0xcd9a70f197238f0ee7045080368dc7d07502b353', (2100000000000000.0, 1))]